In [ ]:
!pip install numpy matplotlib ipympl
# h5py hdf5plugin  scipy


In [ ]:
import sys

In [ ]:
sys.path.append("../build/bliss/python/")

In [ ]:
# import pybland
import pybliss
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
# import h5py
# import hdf5plugin

In [ ]:
data_loc = "/home/nathan/datasets/voyager_2020_data/"
test_file = f"{data_loc}/single_coarse_guppi_59046_80036_DIAG_VOYAGER-1_0011.rawspec.0000.h5"


In [ ]:
help(pybliss.io.h5_filterbank_file)

In [ ]:
filterbank_file = pybliss.io.filterbank_data(test_file)

sec_per_spectra = abs(1 / (1e6*filterbank_file.foff))
N = filterbank_file.tsamp / sec_per_spectra
M = filterbank_file.data.size(0)

sk = pybliss.estimators.spectral_kurtosis(filterbank_file.data, N, M)

plt.figure()
plt.title("Spectral Kurtosis across Channel")
plt.plot(np.from_dlpack(sk))

plt.plot(np.from_dlpack(sk) > 2)
plt.plot(np.from_dlpack(sk) < 0.05)

In [ ]:


##
# TODO: broadcasted copy to store results
##
flagged_filterbank = pybliss.flaggers.flag(filterbank_file, .05, 50)

# plt.figure()
# plt.title("Spectral Kurtosis across Channel")
# plt.plot(np.from_dlpack(sk))